# 내국인 유동인구 제주도지역 전처리
  - 데이터 용량이 너무 커서 resd 제주, 그외 나눠서 전처리

### 1)데이터불러오기

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pops = pd.read_csv('./rowdata/02-1_내국인유동인구_KOREAN.CSV', encoding='CP949')
pops.head()

,base_date,time,city,emd_cd,emd_nm,sex,age,resd_pop_cnt,work_pop_cnt,visit_pop_cnt,resd
0,2018-01-01,1,제주시,50110250,한림읍,여성,0,638.5509,15.5400,151.5149,제주
1,2018-01-01,1,제주시,50110250,한림읍,여성,0,0.0000,3.7249,171.3458,그외
2,2018-01-01,1,제주시,50110250,한림읍,여성,10,668.4577,17.0308,166.0500,제주
3,2018-01-01,1,제주시,50110250,한림읍,여성,10,0.0000,4.2577,195.8539,그외
4,2018-01-01,1,제주시,50110250,한림읍,여성,20,609.7710,5.9490,148.7246,제주


### 2) 제주 거주자 데이터 전처리

In [ ]:
je_pop = pops[pops['resd']=='제주']

In [ ]:
je_pop['emd_nm'].unique()

array(['한림읍', '애월읍', '구좌읍', '조천읍', '한경면', '추자면', '우도면', '일도1동', '일도2동',
       '이도1동', '이도2동', '삼도1동', '삼도2동', '용담1동', '용담2동', '건입동', '화북동',
       '삼양동', '봉개동', '아라동', '오라동', '연동', '노형동', '외도동', '이호동', '도두동',
       '대정읍', '남원읍', '성산읍', '안덕면', '표선면', '송산동', '정방동', '중앙동', '천지동',
       '효돈동', '영천동', '동홍동', '서홍동', '대륜동', '대천동', '중문동', '예래동'],
      dtype=object)

#### 2-0 필요없는 컬럼 삭제

In [ ]:
je_pop= je_pop.loc[:,['base_date','time','emd_nm','sex','age','resd_pop_cnt','work_pop_cnt','visit_pop_cnt']]

In [ ]:
je_pop_g = je_pop.groupby(['base_date','time','emd_nm','sex','age']).sum()
je_pop_g = je_pop_g.reset_index()

In [ ]:
je_pop_g

#### 2-1 제주지역 6개의 구역으로 나누기

In [ ]:
def test(x) :
  if x in ['성산읍','표선면','남원읍'] : return '서귀포시_동'
  elif x in ['대륜동','대천동','동홍동','서홍동','송산동','영천동','예래동','정방동','중문동','중앙동','천지동','효돈동'] : return '서귀포시_중'
  elif x in ['대정읍','안덕면'] : return '서귀포시_서'
  elif x in ['구좌읍','조천읍'] : return '제주시_동'
  elif x in ['건입동','노형동','도두동','봉개동','삼도2동','삼도1동','삼양동','아라동','연동','오라동','외도동','용담2동','용담1동','이도2동','이도1동','이호동','일도2동','일도1동','화북동'] : return '제주시_중'
  elif x in ['애월읍','한림읍','한경면'] : return '제주시_서'

je_pop_g['region'] = je_pop_g['emd_nm'].apply(test)

In [ ]:
je_pop_r = je_pop_g.copy()
je_pop_r

,base_date,time,emd_nm,sex,age,resd_pop_cnt,work_pop_cnt,visit_pop_cnt,region
0,2018-01-01,1,건입동,남성,0,354.2357,3.8557,123.3821,제주시_중
1,2018-01-01,1,건입동,남성,10,446.2578,4.5077,144.2449,제주시_중
2,2018-01-01,1,건입동,남성,20,323.8065,46.2581,132.1659,제주시_중
3,2018-01-01,1,건입동,남성,30,507.2445,44.9457,186.2037,제주시_중
4,2018-01-01,1,건입동,남성,40,691.7851,66.0498,229.4363,제주시_중
...,...,...,...,...,...,...,...,...,...
23721154,2021-06-30,24,효돈동,여성,40,329.2103,0.0000,21.0134,서귀포시_중
23721155,2021-06-30,24,효돈동,여성,50,460.1157,6.1349,24.5395,서귀포시_중
23721156,2021-06-30,24,효돈동,여성,60,394.4375,10.8561,10.8561,서귀포시_중
23721157,2021-06-30,24,효돈동,여성,70,239.4920,0.0000,4.5187,서귀포시_중


#### 2-2 시간대 나누기
 - 낮 (12) : 6:00 ~19:00
 - 밤(24) : 19:00~6:00

In [ ]:
def timetest(x):
     if (x>=6) & (x<=18): return 12
     else : return 24
je_pop_r['time_2'] =  je_pop_r['time'].apply(timetest)

In [ ]:
je_pop_r[je_pop_r['time_2']==24]['time'].unique()

array([ 1,  2,  3,  4,  5, 19, 20, 21, 22, 23, 24], dtype=int64)

In [ ]:
je_pop_12 = je_pop_r[je_pop_r['time_2']==12]
je_pop_24 = je_pop_r[je_pop_r['time_2']==24]
je_pop_12

,base_date,time,emd_nm,sex,age,resd_pop_cnt,work_pop_cnt,visit_pop_cnt,region,time_2
3870,2018-01-01,6,건입동,남성,0,350.7401,3.8566,181.2601,제주시_중,12
3871,2018-01-01,6,건입동,남성,10,441.8541,4.5087,211.9096,제주시_중,12
3872,2018-01-01,6,건입동,남성,20,340.5659,42.9840,198.3879,제주시_중,12
3873,2018-01-01,6,건입동,남성,30,523.8349,32.1371,183.1815,제주시_중,12
3874,2018-01-01,6,건입동,남성,40,696.0316,66.1230,295.8134,제주시_중,12
...,...,...,...,...,...,...,...,...,...,...
23716510,2021-06-30,18,효돈동,여성,40,185.8322,42.0752,112.2006,서귀포시_중,12
23716511,2021-06-30,18,효돈동,여성,50,294.6835,46.0443,113.5759,서귀포시_중,12
23716512,2021-06-30,18,효돈동,여성,60,253.5314,10.8656,123.1438,서귀포시_중,12
23716513,2021-06-30,18,효돈동,여성,70,176.4674,0.0000,36.1984,서귀포시_중,12


#### 2-3 시간대별 dataframe 컬럼명 변경

In [ ]:
je_pop_12r = je_pop_12.rename(columns={'resd_pop_cnt':'n_resd_12','work_pop_cnt':'n_work_12','visit_pop_cnt':'n_visit_12'})
je_pop_12r

,base_date,time,emd_nm,sex,age,n_resd_12,n_work_12,n_visit_12,region,time_2
3870,2018-01-01,6,건입동,남성,0,350.7401,3.8566,181.2601,제주시_중,12
3871,2018-01-01,6,건입동,남성,10,441.8541,4.5087,211.9096,제주시_중,12
3872,2018-01-01,6,건입동,남성,20,340.5659,42.9840,198.3879,제주시_중,12
3873,2018-01-01,6,건입동,남성,30,523.8349,32.1371,183.1815,제주시_중,12
3874,2018-01-01,6,건입동,남성,40,696.0316,66.1230,295.8134,제주시_중,12
...,...,...,...,...,...,...,...,...,...,...
23716510,2021-06-30,18,효돈동,여성,40,185.8322,42.0752,112.2006,서귀포시_중,12
23716511,2021-06-30,18,효돈동,여성,50,294.6835,46.0443,113.5759,서귀포시_중,12
23716512,2021-06-30,18,효돈동,여성,60,253.5314,10.8656,123.1438,서귀포시_중,12
23716513,2021-06-30,18,효돈동,여성,70,176.4674,0.0000,36.1984,서귀포시_중,12


In [ ]:
je_pop_12r[je_pop_12r['region'].isnull()]

,base_date,time,emd_nm,sex,age,n_resd_12,n_work_12,n_visit_12,region,time_2
4338,2018-01-01,6,우도면,남성,0,9.2014,0.0000,34.7094,None,12
4339,2018-01-01,6,우도면,남성,10,9.0174,0.0000,40.5784,None,12
4340,2018-01-01,6,우도면,남성,20,69.4358,6.6129,39.6776,None,12
4341,2018-01-01,6,우도면,남성,30,51.4194,0.0000,54.6331,None,12
4342,2018-01-01,6,우도면,남성,40,125.2857,10.4405,66.1230,None,12
...,...,...,...,...,...,...,...,...,...,...
23716420,2021-06-30,18,추자면,여성,40,87.6567,3.5063,21.0376,None,12
23716421,2021-06-30,18,추자면,여성,50,110.5063,0.0000,27.6266,None,12
23716422,2021-06-30,18,추자면,여성,60,108.6563,0.0000,25.3531,None,12
23716423,2021-06-30,18,추자면,여성,70,113.1201,4.5248,4.5248,None,12


In [ ]:
je_pop_24r = je_pop_24.rename(columns={'resd_pop_cnt':'n_resd_24','work_pop_cnt':'n_work_24','visit_pop_cnt':'n_visit_24'})

#### 2-4 필요없는 컬럼 삭제

In [ ]:
je_pop_24r= je_pop_24r.drop(['time'],axis=1)
je_pop_12r= je_pop_12r.drop(['time'],axis=1)

In [ ]:
je_pop_24r= je_pop_24r.drop(['time_2'],axis=1)
je_pop_12r= je_pop_12r.drop(['time_2'],axis=1)

#### 2-5 각 시간대별로 데이터 저장하기

In [ ]:
je_pop_24r.to_csv("./datas/내국인유동인구_밤시간대_데이터이동용.csv",index=False,encoding='CP949')

In [ ]:
je_pop_12r.to_csv("./datas/내국인유동인구_낮시간대_데이터이동용.csv",index=False,encoding='CP949')

### 3) 밤시간대 데이터 성별및 나이 전처리

In [ ]:
pop_24 =  pd.read_csv('./datas/내국인유동인구_밤시간대_데이터이동용.csv', encoding='CP949')
pop_24.isna().sum()

base_date          0
emd_nm             0
sex                0
age                0
n_resd_24          0
n_work_24          0
n_visit_24         0
region        505508
dtype: int64

In [ ]:
pop_24[pop_24['region'].isnull()]

,base_date,emd_nm,sex,age,n_resd_24,n_work_24,n_visit_24,region
468,2018-01-01,우도면,남성,0,13.7990,0.0000,30.8455,NaN
469,2018-01-01,우도면,남성,10,13.5230,0.0000,36.0612,NaN
470,2018-01-01,우도면,남성,20,62.7788,6.6083,39.6498,NaN
471,2018-01-01,우도면,남성,30,51.3665,0.0000,48.1561,NaN
472,2018-01-01,우도면,남성,40,121.6707,6.9526,59.0972,NaN
...,...,...,...,...,...,...,...,...
10872081,2021-06-30,추자면,여성,40,87.5559,3.5022,3.5022,NaN
10872082,2021-06-30,추자면,여성,50,110.4278,0.0000,15.3372,NaN
10872083,2021-06-30,추자면,여성,60,104.9421,0.0000,25.3309,NaN
10872084,2021-06-30,추자면,여성,70,117.4866,4.5187,4.5187,NaN


#### 3-0 성별나누기

In [ ]:
pop_24_w = pop_24[pop_24['sex']=='여성']
pop_24_m = pop_24[pop_24['sex']=='남성']

In [ ]:
pop_24_w= pop_24_w.drop(['sex'],axis=1)
pop_24_m=pop_24_m.drop(['sex'],axis=1)

#### 3-1 여성 나이대별 컬럼생성
- 0-10대 , 10-20대,  20-30대 : 청년층 ->01020
-  30-40대 , 40-50대,  50-60대 : 중년층 ->304050
- 60-70대 , 70-80대,  80대 이상 : 노년층 ->607080

In [ ]:
pop_24_w_01020= pop_24_w[(pop_24_w['age']==0)|(pop_24_w['age']==10)|(pop_24_w['age']==20)]
pop_24_w_304050= pop_24_w[(pop_24_w['age']==30)|(pop_24_w['age']==40)|(pop_24_w['age']==50)]
pop_24_w_607080= pop_24_w[(pop_24_w['age']==60)|(pop_24_w['age']==70)|(pop_24_w['age']==80)]

In [ ]:
pop_24_w_01020['emd_nm'].unique()

array(['건입동', '구좌읍', '남원읍', '노형동', '대륜동', '대정읍', '대천동', '도두동', '동홍동',
       '봉개동', '삼도1동', '삼도2동', '삼양동', '서홍동', '성산읍', '송산동', '아라동', '안덕면',
       '애월읍', '연동', '영천동', '예래동', '오라동', '외도동', '용담1동', '용담2동', '우도면',
       '이도1동', '이도2동', '이호동', '일도1동', '일도2동', '정방동', '조천읍', '중문동', '중앙동',
       '천지동', '추자면', '표선면', '한경면', '한림읍', '화북동', '효돈동'], dtype=object)

In [ ]:
pop_24_w_01020[pop_24_w_01020['emd_nm']=='일도1동']

,base_date,emd_nm,age,n_resd_24,n_work_24,n_visit_24,region
549,2018-01-01,일도1동,0,49.2676,0.0000,27.1949,제주시_중
550,2018-01-01,일도1동,10,76.6385,0.0000,29.8038,제주시_중
551,2018-01-01,일도1동,20,71.3878,5.9490,47.5919,제주시_중
1323,2018-01-01,일도1동,0,49.2690,0.0000,31.0809,제주시_중
1324,2018-01-01,일도1동,10,76.6407,0.0000,34.0625,제주시_중
...,...,...,...,...,...,...,...
10871178,2021-06-30,일도1동,10,35.4761,0.0000,57.6486,제주시_중
10871179,2021-06-30,일도1동,20,8.3389,5.5592,127.8624,제주시_중
10871951,2021-06-30,일도1동,0,108.4962,0.0000,17.3834,제주시_중
10871952,2021-06-30,일도1동,10,92.9967,0.0000,13.2852,제주시_중


In [ ]:
pop_24_w_01020= pop_24_w_01020.drop(['age'],axis=1)
pop_24_w_304050= pop_24_w_304050.drop(['age'],axis=1)
pop_24_w_607080= pop_24_w_607080.drop(['age'],axis=1)

In [ ]:
pop_24_w_304050

,base_date,emd_nm,n_resd_24,n_work_24,n_visit_24,region
12,2018-01-01,건입동,438.2155,33.9702,125.6897,제주시_중
13,2018-01-01,건입동,709.5961,13.9136,208.7047,제주시_중
14,2018-01-01,건입동,793.9548,50.5244,212.9242,제주시_중
30,2018-01-01,구좌읍,794.9026,27.1762,394.0543,제주시_동
31,2018-01-01,구좌읍,939.1713,10.4352,313.0571,제주시_동
...,...,...,...,...,...,...
10872153,2021-06-30,화북동,2882.3416,49.0313,147.0940,제주시_중
10872154,2021-06-30,화북동,2907.9313,42.9441,141.1022,제주시_중
10872170,2021-06-30,효돈동,228.8828,0.0000,21.9477,서귀포시_중
10872171,2021-06-30,효돈동,329.2103,0.0000,21.0134,서귀포시_중


In [ ]:
g_01020 = pop_24_w_01020.groupby(['base_date','region','emd_nm']).sum()
g_304050 = pop_24_w_304050.groupby(['base_date','region','emd_nm']).sum()
g_607080 = pop_24_w_607080.groupby(['base_date','region','emd_nm']).sum()

In [ ]:
g_607080

n_resd_24  n_work_24  n_visit_24
base_date  region emd_nm                                   
2018-01-01 서귀포시_동 남원읍     17877.4276   165.9416   2053.9942
                  성산읍     16643.7411   134.7935   2288.3799
                  표선면     11953.0293   153.8146   1669.3967
           서귀포시_서 대정읍     23763.6830   454.8418   2280.0025
                  안덕면     12914.8697   222.9044   3297.1231
...                              ...        ...         ...
2021-06-30 제주시_중  이도2동    55313.4543  1633.2513  10762.6049
                  이호동      5379.1671   184.1524   1352.3155
                  일도1동     2995.3500   225.7419   2247.4200
                  일도2동    40767.0986  1205.7847   7203.4171
                  화북동     27231.3537   519.1144   3958.9641

[52357 rows x 3 columns]

In [ ]:
g_01020= g_01020.reset_index()
g_304050 = g_304050.reset_index()
g_607080 = g_607080.reset_index()

In [ ]:
f_24_w_01020 = g_01020.rename(columns={'n_resd_24':'n_resd_24_w_01020','n_work_24':'n_work_24_w_01020','n_visit_24':'n_visit_24_w_01020'})
f_24_w_01020

,base_date,region,emd_nm,n_resd_24_w_01020,n_work_24_w_01020,n_visit_24_w_01020
0,2018-01-01,서귀포시_동,남원읍,14684.5211,566.0329,5139.2220
1,2018-01-01,서귀포시_동,성산읍,14829.6794,742.1910,7717.2921
2,2018-01-01,서귀포시_동,표선면,7730.9421,307.4551,4429.1290
3,2018-01-01,서귀포시_서,대정읍,17495.0165,545.2812,4760.0934
4,2018-01-01,서귀포시_서,안덕면,14655.7078,1722.1888,8070.7980
...,...,...,...,...,...,...
52352,2021-06-30,제주시_중,이도2동,130111.1956,9101.0172,43922.9353
52353,2021-06-30,제주시_중,이호동,5536.1844,147.2096,4359.9596
52354,2021-06-30,제주시_중,일도1동,1909.0860,128.7721,1951.8501
52355,2021-06-30,제주시_중,일도2동,62874.5647,2804.9946,14488.9310


In [ ]:
f_24_w_304050 = g_304050.rename(columns={'n_resd_24':'n_resd_24_w_304050','n_work_24':'n_work_24_w_304050','n_visit_24':'n_visit_24_w_304050'})
f_24_w_607080 = g_607080.rename(columns={'n_resd_24':'n_resd_24_w_607080','n_work_24':'n_work_24_w_607080','n_visit_24':'n_visit_24_w_607080'})

In [ ]:
f_24_w_304050

,base_date,region,emd_nm,n_resd_24_w_304050,n_work_24_w_304050,n_visit_24_w_304050
0,2018-01-01,서귀포시_동,남원읍,30916.7317,909.5769,6540.5814
1,2018-01-01,서귀포시_동,성산읍,29849.8498,1020.5545,8790.3556
2,2018-01-01,서귀포시_동,표선면,22582.4659,756.7657,5185.7867
3,2018-01-01,서귀포시_서,대정읍,39764.7801,999.5837,6984.4644
4,2018-01-01,서귀포시_서,안덕면,25031.3898,1246.0013,8651.1944
...,...,...,...,...,...,...
52352,2021-06-30,제주시_중,이도2동,143830.2800,8822.4937,33772.4080
52353,2021-06-30,제주시_중,이호동,8984.1205,392.9849,4582.0540
52354,2021-06-30,제주시_중,일도1동,2869.4610,543.5638,3294.3765
52355,2021-06-30,제주시_중,일도2동,78366.5728,3155.8503,17046.7279


#### 3-2 남성 나이대별 컬럼생성
- 0-10대 , 10-20대,  20-30대 : 청년층 ->01020
-  30-40대 , 40-50대,  50-60대 : 중년층 ->304050
- 60-70대 , 70-80대,  80대 이상 : 노년층 ->607080

In [ ]:
pop_24_m_01020= pop_24_m[(pop_24_m['age']==0)|(pop_24_m['age']==10)|(pop_24_m['age']==20)]
pop_24_m_304050= pop_24_m[(pop_24_m['age']==30)|(pop_24_m['age']==40)|(pop_24_m['age']==50)]
pop_24_m_607080= pop_24_m[(pop_24_m['age']==60)|(pop_24_m['age']==70)|(pop_24_m['age']==80)]

In [ ]:
pop_24_m_01020

,base_date,emd_nm,age,n_resd_24,n_work_24,n_visit_24,region
0,2018-01-01,건입동,0,354.2357,3.8557,123.3821,제주시_중
1,2018-01-01,건입동,10,446.2578,4.5077,144.2449,제주시_중
2,2018-01-01,건입동,20,323.8065,46.2581,132.1659,제주시_중
18,2018-01-01,구좌읍,0,406.3413,11.5671,196.6403,제주시_동
19,2018-01-01,구좌읍,10,468.7960,13.5230,229.8904,제주시_동
...,...,...,...,...,...,...,...
10872141,2021-06-30,화북동,10,2445.9602,27.0771,194.0522,제주시_중
10872142,2021-06-30,화북동,20,1383.1928,41.9998,198.7990,제주시_중
10872158,2021-06-30,효돈동,0,221.5921,3.9153,15.6615,서귀포시_중
10872159,2021-06-30,효돈동,10,248.2063,4.5128,18.0514,서귀포시_중


In [ ]:
pop_24_m_01020= pop_24_m_01020.drop(['age'],axis=1)
pop_24_m_304050= pop_24_m_304050.drop(['age'],axis=1)
pop_24_m_607080= pop_24_m_607080.drop(['age'],axis=1)

In [ ]:
g_01020 = pop_24_m_01020.groupby(['base_date','region','emd_nm']).sum()
g_304050 = pop_24_m_304050.groupby(['base_date','region','emd_nm']).sum()
g_607080 = pop_24_m_607080.groupby(['base_date','region','emd_nm']).sum()

In [ ]:
g_01020 = g_01020.reset_index()
g_304050 = g_304050.reset_index()
g_607080 = g_607080.reset_index()

In [ ]:
f_24_m_01020 = g_01020.rename(columns={'n_resd_24':'n_resd_24_m_01020','n_work_24':'n_work_24_m_01020','n_visit_24':'n_visit_24_m_01020'})
f_24_m_01020

,index,base_date,region,emd_nm,n_resd_24_m_01020,n_work_24_m_01020,n_visit_24_m_01020
0,0,2018-01-01,서귀포시_동,남원읍,17803.0511,490.9392,5466.4894
1,1,2018-01-01,서귀포시_동,성산읍,15068.5259,352.4037,7137.4200
2,2,2018-01-01,서귀포시_동,표선면,10624.6740,428.6640,3078.3089
3,3,2018-01-01,서귀포시_서,대정읍,22458.4297,803.5026,4916.7731
4,4,2018-01-01,서귀포시_서,안덕면,15048.2112,1394.3328,6746.4510
...,...,...,...,...,...,...,...
52352,52352,2021-06-30,제주시_중,이도2동,110940.4282,5617.9672,39269.4743
52353,52353,2021-06-30,제주시_중,이호동,5104.2802,168.3704,4580.8548
52354,52354,2021-06-30,제주시_중,일도1동,1744.7538,151.6659,2183.3393
52355,52355,2021-06-30,제주시_중,일도2동,54073.5195,1992.6164,13891.6925


In [ ]:
f_24_m_304050 = g_304050.rename(columns={'n_resd_24':'n_resd_24_m_304050','n_work_24':'n_work_24_m_304050','n_visit_24':'n_visit_24_m_304050'})
f_24_m_607080 = g_607080.rename(columns={'n_resd_24':'n_resd_24_m_607080','n_work_24':'n_work_24_m_607080','n_visit_24':'n_visit_24_m_607080'})

In [ ]:
f_24_m_304050

,base_date,region,emd_nm,n_resd_24_m_304050,n_work_24_m_304050,n_visit_24_m_304050
0,2018-01-01,서귀포시_동,남원읍,37964.1653,1291.2687,7806.0479
1,2018-01-01,서귀포시_동,성산읍,39374.1297,1884.8375,10411.3381
2,2018-01-01,서귀포시_동,표선면,26516.8208,1105.8330,6666.7966
3,2018-01-01,서귀포시_서,대정읍,46811.6633,1199.0711,8170.6528
4,2018-01-01,서귀포시_서,안덕면,30815.8761,2924.9774,10374.2006
...,...,...,...,...,...,...
52352,2021-06-30,제주시_중,이도2동,126545.3244,9027.8786,37007.4136
52353,2021-06-30,제주시_중,이호동,9015.0224,606.3196,5947.7833
52354,2021-06-30,제주시_중,일도1동,3699.2967,438.6033,3922.8996
52355,2021-06-30,제주시_중,일도2동,68013.4037,2237.8730,17918.7016


### 4) 낮 시간대 데이터 성별 및 나이 전처리

In [ ]:
pop_12 =  pd.read_csv('./datas/내국인유동인구_낮시간대_데이터이동용.csv', encoding='CP949')
pop_12.head()

,base_date,emd_nm,sex,age,n_resd_12,n_work_12,n_visit_12,region
0,2018-01-01,건입동,남성,0,350.7401,3.8566,181.2601,제주시_중
1,2018-01-01,건입동,남성,10,441.8541,4.5087,211.9096,제주시_중
2,2018-01-01,건입동,남성,20,340.5659,42.9840,198.3879,제주시_중
3,2018-01-01,건입동,남성,30,523.8349,32.1371,183.1815,제주시_중
4,2018-01-01,건입동,남성,40,696.0316,66.1230,295.8134,제주시_중


In [ ]:
pop_12['emd_nm'].unique()

array(['건입동', '구좌읍', '남원읍', '노형동', '대륜동', '대정읍', '대천동', '도두동', '동홍동',
       '봉개동', '삼도1동', '삼도2동', '삼양동', '서홍동', '성산읍', '송산동', '아라동', '안덕면',
       '애월읍', '연동', '영천동', '예래동', '오라동', '외도동', '용담1동', '용담2동', '우도면',
       '이도1동', '이도2동', '이호동', '일도1동', '일도2동', '정방동', '조천읍', '중문동', '중앙동',
       '천지동', '추자면', '표선면', '한경면', '한림읍', '화북동', '효돈동'], dtype=object)

#### 4-0 성별나누기

In [ ]:
pop_12_w = pop_12[pop_12['sex']=='여성']
pop_12_m = pop_12[pop_12['sex']=='남성']

In [ ]:
pop_12_w= pop_12_w.drop(['sex'],axis=1)
pop_12_m=pop_12_m.drop(['sex'],axis=1)

#### 4-1 여성 나이대별 컬럼생성
- 0-10대 , 10-20대,  20-30대 : 청년층 ->01020
-  30-40대 , 40-50대,  50-60대 : 중년층 ->304050
- 60-70대 , 70-80대,  80대 이상 : 노년층 ->607080

In [ ]:
pop_12_w_01020= pop_12_w[(pop_12_w['age']==0)|(pop_12_w['age']==10)|(pop_12_w['age']==20)]
pop_12_w_304050= pop_12_w[(pop_12_w['age']==30)|(pop_12_w['age']==40)|(pop_12_w['age']==50)]
pop_12_w_607080= pop_12_w[(pop_12_w['age']==60)|(pop_12_w['age']==70)|(pop_12_w['age']==80)]

In [ ]:
pop_12_w_304050

,base_date,emd_nm,age,n_resd_12,n_work_12,n_visit_12,region
12,2018-01-01,건입동,30,448.6116,20.3914,135.9429,제주시_중
13,2018-01-01,건입동,40,730.8480,6.9605,219.2544,제주시_중
14,2018-01-01,건입동,50,776.1442,14.4399,231.0383,제주시_중
30,2018-01-01,구좌읍,30,788.4688,16.9929,431.6187,제주시_동
31,2018-01-01,구좌읍,40,946.6222,10.4407,407.1868,제주시_동
...,...,...,...,...,...,...,...
12848960,2021-06-30,화북동,40,1630.4148,308.5516,950.1987,제주시_중
12848961,2021-06-30,화북동,50,1712.8476,245.5695,847.2149,제주시_중
12848977,2021-06-30,효돈동,30,128.7046,12.5566,65.9219,서귀포시_중
12848978,2021-06-30,효돈동,40,185.8322,42.0752,112.2006,서귀포시_중


In [ ]:
pop_12_w_01020= pop_12_w_01020.drop(['age'],axis=1)
pop_12_w_304050= pop_12_w_304050.drop(['age'],axis=1)
pop_12_w_607080= pop_12_w_607080.drop(['age'],axis=1)

In [ ]:
g_01020 = pop_12_w_01020.groupby(['base_date','region','emd_nm']).sum()
g_304050 = pop_12_w_304050.groupby(['base_date','region','emd_nm']).sum()
g_607080 = pop_12_w_607080.groupby(['base_date','region','emd_nm']).sum()

In [ ]:
g_01020 = g_01020.reset_index()
g_304050 = g_304050.reset_index()
g_607080 = g_607080.reset_index()

In [ ]:
g_01020['emd_nm'].unique()

array(['남원읍', '성산읍', '표선면', '대정읍', '안덕면', '대륜동', '대천동', '동홍동', '서홍동',
       '송산동', '영천동', '예래동', '정방동', '중문동', '중앙동', '천지동', '효돈동', '구좌읍',
       '조천읍', '애월읍', '한경면', '한림읍', '건입동', '노형동', '도두동', '봉개동', '삼도1동',
       '삼도2동', '삼양동', '아라동', '연동', '오라동', '외도동', '용담1동', '용담2동', '이도1동',
       '이도2동', '이호동', '일도1동', '일도2동', '화북동'], dtype=object)

In [ ]:
f_12_w_01020= g_01020.rename(columns={'n_resd_12':'n_resd_12_w_01020','n_work_12':'n_work_12_w_01020','n_visit_12':'n_visit_12_w_01020'})
f_12_w_01020

,base_date,region,emd_nm,n_resd_12_w_01020,n_work_12_w_01020,n_visit_12_w_01020
0,2018-01-01,서귀포시_동,남원읍,16153.5654,812.6967,11107.5655
1,2018-01-01,서귀포시_동,성산읍,16727.8881,1250.9516,14703.8154
2,2018-01-01,서귀포시_동,표선면,8873.5163,568.9623,6859.5800
3,2018-01-01,서귀포시_서,대정읍,19245.1729,781.7883,8205.6476
4,2018-01-01,서귀포시_서,안덕면,16913.5385,3380.7731,14187.6894
...,...,...,...,...,...,...
52352,2021-06-30,제주시_중,이도2동,122657.0680,21701.3394,69833.8889
52353,2021-06-30,제주시_중,이호동,5090.7488,535.4197,3648.4354
52354,2021-06-30,제주시_중,일도1동,1270.1962,237.9582,3400.2110
52355,2021-06-30,제주시_중,일도2동,54582.1853,4274.7014,21478.8876


In [ ]:
f_12_w_304050 = g_304050.rename(columns={'n_resd_12':'n_resd_12_w_304050','n_work_12':'n_work_12_w_304050','n_visit_12':'n_visit_12_w_304050'})
f_12_w_607080 = g_607080.rename(columns={'n_resd_12':'n_resd_12_w_607080','n_work_12':'n_work_12_w_607080','n_visit_12':'n_visit_12_w_607080'})

In [ ]:
f_24_w_304050

,base_date,region,emd_nm,n_resd_24_w_304050,n_work_24_w_304050,n_visit_24_w_304050
0,2018-01-01,서귀포시_동,남원읍,30916.7317,909.5769,6540.5814
1,2018-01-01,서귀포시_동,성산읍,29849.8498,1020.5545,8790.3556
2,2018-01-01,서귀포시_동,표선면,22582.4659,756.7657,5185.7867
3,2018-01-01,서귀포시_서,대정읍,39764.7801,999.5837,6984.4644
4,2018-01-01,서귀포시_서,안덕면,25031.3898,1246.0013,8651.1944
...,...,...,...,...,...,...
52352,2021-06-30,제주시_중,이도2동,143830.2800,8822.4937,33772.4080
52353,2021-06-30,제주시_중,이호동,8984.1205,392.9849,4582.0540
52354,2021-06-30,제주시_중,일도1동,2869.4610,543.5638,3294.3765
52355,2021-06-30,제주시_중,일도2동,78366.5728,3155.8503,17046.7279


#### 4-2 남성 나이대별 컬럼생성
- 0-10대 , 10-20대,  20-30대 : 청년층 ->01020
-  30-40대 , 40-50대,  50-60대 : 중년층 ->304050
- 60-70대 , 70-80대,  80대 이상 : 노년층 ->607080

In [ ]:
pop_12_m_01020= pop_12_m[(pop_12_m['age']==0)|(pop_12_m['age']==10)|(pop_12_m['age']==20)]
pop_12_m_304050= pop_12_m[(pop_12_m['age']==30)|(pop_12_m['age']==40)|(pop_12_m['age']==50)]
pop_12_m_607080= pop_12_m[(pop_12_m['age']==60)|(pop_12_m['age']==70)|(pop_12_m['age']==80)]

In [ ]:
pop_12_m_304050

,base_date,emd_nm,age,n_resd_12,n_work_12,n_visit_12,region
3,2018-01-01,건입동,30,523.8349,32.1371,183.1815,제주시_중
4,2018-01-01,건입동,40,696.0316,66.1230,295.8134,제주시_중
5,2018-01-01,건입동,50,749.6382,77.9327,356.2637,제주시_중
21,2018-01-01,구좌읍,30,703.8028,41.7782,324.5848,제주시_동
22,2018-01-01,구좌읍,40,1266.7775,55.6825,473.3015,제주시_동
...,...,...,...,...,...,...,...
12848951,2021-06-30,화북동,40,1304.3827,392.6127,937.7279,제주시_중
12848952,2021-06-30,화북동,50,1537.5923,337.4426,945.4760,제주시_중
12848968,2021-06-30,효돈동,30,122.2271,25.5824,73.9048,서귀포시_중
12848969,2021-06-30,효돈동,40,233.6208,35.6921,107.0762,서귀포시_중


In [ ]:
pop_12_m_01020= pop_12_m_01020.drop(['age'],axis=1)
pop_12_m_304050= pop_12_m_304050.drop(['age'],axis=1)
pop_12_m_607080= pop_12_m_607080.drop(['age'],axis=1)

In [ ]:
g_01020 = pop_12_m_01020.groupby(['base_date','region','emd_nm']).sum()
g_304050 = pop_12_m_304050.groupby(['base_date','region','emd_nm']).sum()
g_607080 = pop_12_m_607080.groupby(['base_date','region','emd_nm']).sum()

In [ ]:
g_01020 = g_01020.reset_index()
g_304050 = g_304050.reset_index()
g_607080 = g_607080.reset_index()

In [ ]:
f_12_m_01020 = g_01020.rename(columns={'n_resd_12':'n_resd_12_m_01020','n_work_12':'n_work_12_m_01020','n_visit_12':'n_visit_12_m_01020'})
f_12_m_01020

,base_date,region,emd_nm,n_resd_12_m_01020,n_work_12_m_01020,n_visit_12_m_01020
0,2018-01-01,서귀포시_동,남원읍,19551.2075,998.2235,11951.1205
1,2018-01-01,서귀포시_동,성산읍,16652.6077,863.4747,14905.7896
2,2018-01-01,서귀포시_동,표선면,11639.7076,896.8565,6581.0867
3,2018-01-01,서귀포시_서,대정읍,26000.9456,992.0483,8203.5551
4,2018-01-01,서귀포시_서,안덕면,17267.1705,2603.6085,11333.9277
...,...,...,...,...,...,...
52352,2021-06-30,제주시_중,이도2동,105821.3438,11164.0412,49292.4264
52353,2021-06-30,제주시_중,이호동,4210.0467,662.1842,4011.1319
52354,2021-06-30,제주시_중,일도1동,1464.8202,235.6243,3098.1344
52355,2021-06-30,제주시_중,일도2동,46994.4603,2504.5758,14945.7378


In [ ]:
f_12_m_304050 = g_304050.rename(columns={'n_resd_12':'n_resd_12_m_304050','n_work_12':'n_work_12_m_304050','n_visit_12':'n_visit_12_m_304050'})
f_12_m_607080 = g_607080.rename(columns={'n_resd_12':'n_resd_12_m_607080','n_work_12':'n_work_12_m_607080','n_visit_12':'n_visit_12_m_607080'})

In [ ]:
f_12_m_304050

,base_date,region,emd_nm,n_resd_12_m_304050,n_work_12_m_304050,n_visit_12_m_304050
0,2018-01-01,서귀포시_동,남원읍,40878.1785,3923.7522,23220.8175
1,2018-01-01,서귀포시_동,성산읍,43521.3791,3686.8327,26526.2522
2,2018-01-01,서귀포시_동,표선면,28059.1008,2421.2159,15794.0004
3,2018-01-01,서귀포시_서,대정읍,50768.3756,2393.0628,19205.2393
4,2018-01-01,서귀포시_서,안덕면,32752.9210,6468.8658,24651.9595
...,...,...,...,...,...,...
52352,2021-06-30,제주시_중,이도2동,94907.2536,28374.3655,59763.4002
52353,2021-06-30,제주시_중,이호동,6762.7915,2196.7090,10364.8247
52354,2021-06-30,제주시_중,일도1동,2950.6077,1144.0477,6631.9541
52355,2021-06-30,제주시_중,일도2동,45158.7658,5851.4490,24838.9743


### 5) 각 시간대별 성별, 나이별 dataframe merge

In [ ]:
merge_data = pd.merge(f_12_w_01020,f_12_w_304050, how='outer',on=['base_date','region','emd_nm'])

In [ ]:
merge_data = pd.merge(merge_data,f_12_w_607080, how='outer',on=['base_date','region','emd_nm'])

In [ ]:
merge_data = pd.merge(merge_data,f_12_m_01020, how='outer',on=['base_date','region','emd_nm'])

In [ ]:
merge_data = pd.merge(merge_data,f_12_m_304050, how='outer',on=['base_date','region','emd_nm'])

In [ ]:
merge_data = pd.merge(merge_data,f_12_m_607080, how='outer',on=['base_date','region','emd_nm'])

In [ ]:
merge_data = pd.merge(merge_data,f_24_w_01020, how='outer',on=['base_date','region','emd_nm'])

In [ ]:
merge_data = pd.merge(merge_data,f_24_w_304050, how='outer',on=['base_date','region','emd_nm'])

In [ ]:
merge_data = pd.merge(merge_data,f_24_w_607080, how='outer',on=['base_date','region','emd_nm'])

In [ ]:
merge_data = pd.merge(merge_data,f_24_m_01020, how='outer',on=['base_date','region','emd_nm'])

In [ ]:
merge_data = pd.merge(merge_data,f_24_m_304050, how='outer',on=['base_date','region','emd_nm'])

In [ ]:
merge_data = pd.merge(merge_data,f_24_m_607080, how='outer',on=['base_date','region','emd_nm'])

In [ ]:
merge_data

,base_date,region,emd_nm,n_resd_12_w_01020,n_work_12_w_01020,n_visit_12_w_01020,n_resd_12_w_304050,n_work_12_w_304050,n_visit_12_w_304050
0,2018-01-01,서귀포시_동,남원읍,16153.5654,812.6967,11107.5655,32585.7557,2484.3162,19002.9037
1,2018-01-01,서귀포시_동,성산읍,16727.8881,1250.9516,14703.8154,33126.8217,2341.8504,22326.8821
2,2018-01-01,서귀포시_동,표선면,8873.5163,568.9623,6859.5800,24223.4533,2424.2823,12676.6533
3,2018-01-01,서귀포시_서,대정읍,19245.1729,781.7883,8205.6476,42315.9377,1907.2044,15550.1501
4,2018-01-01,서귀포시_서,안덕면,16913.5385,3380.7731,14187.6894,26786.2950,3727.2655,21288.5303
...,...,...,...,...,...,...,...,...,...
52352,2021-06-30,제주시_중,이도2동,122657.0680,21701.3394,69833.8889,121049.8113,31516.5995,64941.3425
52353,2021-06-30,제주시_중,이호동,5090.7488,535.4197,3648.4354,6918.4482,1456.7255,7537.4279
52354,2021-06-30,제주시_중,일도1동,1270.1962,237.9582,3400.2110,2257.7188,1419.0302,7465.5576
52355,2021-06-30,제주시_중,일도2동,54582.1853,4274.7014,21478.8876,60217.6651,7175.8878,26276.1253


### 6)제주 지역 데이터 저장

In [ ]:
merge_data.to_csv("./datas/neddata_내국인유동인구_제주지역.csv", index=False, encoding='CP949')

# 내국인 유동인구 그외 지역 전처리

### 1)데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pops = pd.read_csv('./rowdata/02-1_내국인유동인구_KOREAN.CSV', encoding='CP949')
pops.head()

,base_date,time,city,emd_cd,emd_nm,sex,age,resd_pop_cnt,work_pop_cnt,visit_pop_cnt,resd
0,2018-01-01,1,제주시,50110250,한림읍,여성,0,638.5509,15.5400,151.5149,제주
1,2018-01-01,1,제주시,50110250,한림읍,여성,0,0.0000,3.7249,171.3458,그외
2,2018-01-01,1,제주시,50110250,한림읍,여성,10,668.4577,17.0308,166.0500,제주
3,2018-01-01,1,제주시,50110250,한림읍,여성,10,0.0000,4.2577,195.8539,그외
4,2018-01-01,1,제주시,50110250,한림읍,여성,20,609.7710,5.9490,148.7246,제주


### 2) 그외 지역 거주자 데이터 전처리

In [ ]:
je_pop = pops[pops['resd']=='그외']

In [ ]:
je_pop['emd_nm'].unique()

array(['한림읍', '애월읍', '구좌읍', '조천읍', '한경면', '추자면', '우도면', '일도1동', '일도2동',
       '이도1동', '이도2동', '삼도1동', '삼도2동', '용담1동', '용담2동', '건입동', '화북동',
       '삼양동', '봉개동', '아라동', '오라동', '연동', '노형동', '외도동', '이호동', '도두동',
       '대정읍', '남원읍', '성산읍', '안덕면', '표선면', '송산동', '정방동', '중앙동', '천지동',
       '효돈동', '영천동', '동홍동', '서홍동', '대륜동', '대천동', '중문동', '예래동'],
      dtype=object)

#### 2-0 필요없는 컬럼 삭제

In [ ]:
je_pop= je_pop.loc[:,['base_date','time','emd_nm','sex','age','resd_pop_cnt','work_pop_cnt','visit_pop_cnt']]

In [ ]:
je_pop_g = je_pop.groupby(['base_date','time','emd_nm','sex','age']).sum()
je_pop_g = je_pop_g.reset_index()

#### 2-1 제주지역 6개의 구역으로 나누기

In [ ]:
def test(x) :
  if x in ['성산읍','표선면','남원읍'] : return '서귀포시_동'
  elif x in ['대륜동','대천동','동홍동','서홍동','송산동','영천동','예래동','정방동','중문동','중앙동','천지동','효돈동'] : return '서귀포시_중'
  elif x in ['대정읍','안덕면'] : return '서귀포시_서'
  elif x in ['구좌읍','조천읍'] : return '제주시_동'
  elif x in ['건입동','노형동','도두동','봉개동','삼도2동','삼도1동','삼양동','아라동','연동','오라동','외도동','용담2동','용담1동','이도2동','이도1동','이호동','일도2동','일도1동','화북동'] : return '제주시_중'
  elif x in ['애월읍','한림읍','한경면'] : return '제주시_서'

je_pop_g['region'] = je_pop_g['emd_nm'].apply(test)

In [ ]:
je_pop_r = je_pop_g.copy()
je_pop_r

,base_date,time,emd_nm,sex,age,resd_pop_cnt,work_pop_cnt,visit_pop_cnt,region
0,2018-01-01,1,건입동,남성,0,0.0,3.7485,112.4523,제주시_중
1,2018-01-01,1,건입동,남성,10,0.0,4.5077,135.2296,제주시_중
2,2018-01-01,1,건입동,남성,20,0.0,13.2166,287.4608,제주시_중
3,2018-01-01,1,건입동,남성,30,0.0,6.4208,237.5702,제주시_중
4,2018-01-01,1,건입동,남성,40,0.0,10.4289,229.4363,제주시_중
...,...,...,...,...,...,...,...,...,...
23529032,2021-06-30,24,효돈동,여성,40,0.0,0.0000,28.0179,서귀포시_중
23529033,2021-06-30,24,효돈동,여성,50,0.0,0.0000,39.8767,서귀포시_중
23529034,2021-06-30,24,효돈동,여성,60,0.0,0.0000,28.9495,서귀포시_중
23529035,2021-06-30,24,효돈동,여성,70,0.0,4.5187,18.0749,서귀포시_중


#### 2-2 시간대 나누기

In [ ]:
def timetest(x):
     if (x>=6) & (x<=18): return 12
     else : return 24
je_pop_r['time_2'] =  je_pop_r['time'].apply(timetest)

In [ ]:
je_pop_r[je_pop_r['time_2']==24]['time'].unique()

array([ 1,  2,  3,  4,  5, 19, 20, 21, 22, 23, 24], dtype=int64)

In [ ]:
je_pop_12 = je_pop_r[je_pop_r['time_2']==12]
je_pop_24 = je_pop_r[je_pop_r['time_2']==24]
je_pop_12

,base_date,time,emd_nm,sex,age,resd_pop_cnt,work_pop_cnt,visit_pop_cnt,region,time_2
3834,2018-01-01,6,건입동,남성,0,0.0,3.7493,108.7295,제주시_중,12
3835,2018-01-01,6,건입동,남성,10,0.0,4.5087,130.7527,제주시_중,12
3836,2018-01-01,6,건입동,남성,20,0.0,9.9194,267.8237,제주시_중,12
3837,2018-01-01,6,건입동,남성,30,0.0,3.2137,241.0283,제주시_중,12
3838,2018-01-01,6,건입동,남성,40,0.0,10.4405,236.6507,제주시_중,12
...,...,...,...,...,...,...,...,...,...,...
23524390,2021-06-30,18,효돈동,여성,40,0.0,0.0000,59.6066,서귀포시_중,12
23524391,2021-06-30,18,효돈동,여성,50,0.0,0.0000,73.6709,서귀포시_중,12
23524392,2021-06-30,18,효돈동,여성,60,0.0,0.0000,47.0844,서귀포시_중,12
23524393,2021-06-30,18,효돈동,여성,70,0.0,4.5248,18.0992,서귀포시_중,12


#### 2-3 시간대별 dataframe 컬럼명 변경

In [ ]:
je_pop_12r = je_pop_12.rename(columns={'resd_pop_cnt':'n_e_resd_12','work_pop_cnt':'n_e_work_12','visit_pop_cnt':'n_e_visit_12'})
je_pop_12r

,base_date,time,emd_nm,sex,age,n_e_resd_12,n_e_work_12,n_e_visit_12,region,time_2
3834,2018-01-01,6,건입동,남성,0,0.0,3.7493,108.7295,제주시_중,12
3835,2018-01-01,6,건입동,남성,10,0.0,4.5087,130.7527,제주시_중,12
3836,2018-01-01,6,건입동,남성,20,0.0,9.9194,267.8237,제주시_중,12
3837,2018-01-01,6,건입동,남성,30,0.0,3.2137,241.0283,제주시_중,12
3838,2018-01-01,6,건입동,남성,40,0.0,10.4405,236.6507,제주시_중,12
...,...,...,...,...,...,...,...,...,...,...
23524390,2021-06-30,18,효돈동,여성,40,0.0,0.0000,59.6066,서귀포시_중,12
23524391,2021-06-30,18,효돈동,여성,50,0.0,0.0000,73.6709,서귀포시_중,12
23524392,2021-06-30,18,효돈동,여성,60,0.0,0.0000,47.0844,서귀포시_중,12
23524393,2021-06-30,18,효돈동,여성,70,0.0,4.5248,18.0992,서귀포시_중,12


In [ ]:
je_pop_24r = je_pop_24.rename(columns={'resd_pop_cnt':'n_e_resd_24','work_pop_cnt':'n_e_work_24','visit_pop_cnt':'n_e_visit_24'})

#### 2-4 필요없는 컬럼 삭제

In [ ]:
je_pop_24r= je_pop_24r.drop(['time'],axis=1)
je_pop_12r= je_pop_12r.drop(['time'],axis=1)

In [ ]:
je_pop_24r= je_pop_24r.drop(['time_2'],axis=1)
je_pop_12r= je_pop_12r.drop(['time_2'],axis=1)

#### 2-5 각 시간대별로 데이터 저장하기

In [ ]:
je_pop_24r.to_csv("./datas/내국인유동인구_그외_밤시간대_데이터이동용.csv",index=False,encoding='CP949')

In [ ]:
je_pop_12r.to_csv("./datas/내국인유동인구_그외_낮시간대_데이터이동용.csv",index=False,encoding='CP949')

### 3) 밤 시간대 데이터 성별 및 나이 전처리

In [ ]:
pop_24 =  pd.read_csv('./datas/내국인유동인구_그외_밤시간대_데이터이동용.csv', encoding='CP949')
pop_24.isna().sum()

base_date            0
emd_nm               0
sex                  0
age                  0
n_e_resd_24          0
n_e_work_24          0
n_e_visit_24         0
region          471002
dtype: int64

In [ ]:
pop_24[pop_24['region'].isnull()]

,base_date,emd_nm,sex,age,n_e_resd_24,n_e_work_24,n_e_visit_24,region
466,2018-01-01,우도면,남성,0,0.0,0.0000,3.7485,NaN
467,2018-01-01,우도면,남성,10,0.0,0.0000,4.5077,NaN
468,2018-01-01,우도면,남성,20,0.0,0.0000,46.2581,NaN
469,2018-01-01,우도면,남성,30,0.0,0.0000,38.5249,NaN
470,2018-01-01,우도면,남성,40,0.0,3.4763,24.3341,NaN
...,...,...,...,...,...,...,...,...
10774772,2021-06-30,추자면,여성,40,0.0,3.5022,14.0090,NaN
10774773,2021-06-30,추자면,여성,50,0.0,0.0000,33.7418,NaN
10774774,2021-06-30,추자면,여성,60,0.0,0.0000,50.6617,NaN
10774775,2021-06-30,추자면,여성,70,0.0,0.0000,22.5936,NaN


####  3-0 성별 나누기

In [ ]:
pop_24_w = pop_24[pop_24['sex']=='여성']
pop_24_m = pop_24[pop_24['sex']=='남성']

In [ ]:
pop_24_w= pop_24_w.drop(['sex'],axis=1)
pop_24_m=pop_24_m.drop(['sex'],axis=1)

#### 3-1 여성 나이대별 컬럼생성
- 0-10대 , 10-20대,  20-30대 : 청년층 ->01020
-  30-40대 , 40-50대,  50-60대 : 중년층 ->304050
- 60-70대 , 70-80대,  80대 이상 : 노년층 ->607080

In [ ]:
pop_24_w_01020= pop_24_w[(pop_24_w['age']==0)|(pop_24_w['age']==10)|(pop_24_w['age']==20)]
pop_24_w_304050= pop_24_w[(pop_24_w['age']==30)|(pop_24_w['age']==40)|(pop_24_w['age']==50)]
pop_24_w_607080= pop_24_w[(pop_24_w['age']==60)|(pop_24_w['age']==70)|(pop_24_w['age']==80)]

In [ ]:
pop_24_w_01020['emd_nm'].unique()

array(['건입동', '구좌읍', '남원읍', '노형동', '대륜동', '대정읍', '대천동', '도두동', '동홍동',
       '봉개동', '삼도1동', '삼도2동', '삼양동', '서홍동', '성산읍', '송산동', '아라동', '안덕면',
       '애월읍', '연동', '영천동', '예래동', '오라동', '외도동', '용담1동', '용담2동', '우도면',
       '이도1동', '이도2동', '이호동', '일도1동', '일도2동', '정방동', '조천읍', '중문동', '중앙동',
       '천지동', '추자면', '표선면', '한경면', '한림읍', '화북동', '효돈동'], dtype=object)

In [ ]:
pop_24_w_01020[pop_24_w_01020['emd_nm']=='일도1동']

,base_date,emd_nm,age,n_e_resd_24,n_e_work_24,n_e_visit_24,region
545,2018-01-01,일도1동,0,0.0,0.0,11.1747,제주시_중
546,2018-01-01,일도1동,10,0.0,0.0,12.7731,제주시_중
547,2018-01-01,일도1동,20,0.0,0.0,17.8470,제주시_중
1312,2018-01-01,일도1동,0,0.0,0.0,11.1751,제주시_중
1313,2018-01-01,일도1동,10,0.0,0.0,12.7735,제주시_중
...,...,...,...,...,...,...,...
10773871,2021-06-30,일도1동,10,0.0,0.0,39.9106,제주시_중
10773872,2021-06-30,일도1동,20,0.0,0.0,44.4739,제주시_중
10774644,2021-06-30,일도1동,0,0.0,0.0,39.3321,제주시_중
10774645,2021-06-30,일도1동,10,0.0,0.0,30.9989,제주시_중


In [ ]:
pop_24_w_01020= pop_24_w_01020.drop(['age'],axis=1)
pop_24_w_304050= pop_24_w_304050.drop(['age'],axis=1)
pop_24_w_607080= pop_24_w_607080.drop(['age'],axis=1)

In [ ]:
pop_24_w_304050

,base_date,emd_nm,n_e_resd_24,n_e_work_24,n_e_visit_24,region
12,2018-01-01,건입동,0.0,0.0000,234.3944,제주시_중
13,2018-01-01,건입동,0.0,3.4784,274.7946,제주시_중
14,2018-01-01,건입동,0.0,10.8267,198.4887,제주시_중
30,2018-01-01,구좌읍,0.0,0.0000,601.2725,제주시_동
31,2018-01-01,구좌읍,0.0,10.4352,406.9742,제주시_동
...,...,...,...,...,...,...
10774844,2021-06-30,화북동,0.0,17.5112,108.5694,제주시_중
10774845,2021-06-30,화북동,0.0,6.1349,76.6860,제주시_중
10774861,2021-06-30,효돈동,0.0,0.0000,50.1661,서귀포시_중
10774862,2021-06-30,효돈동,0.0,0.0000,28.0179,서귀포시_중


In [ ]:
g_01020 = pop_24_w_01020.groupby(['base_date','region','emd_nm']).sum()
g_304050 = pop_24_w_304050.groupby(['base_date','region','emd_nm']).sum()
g_607080 = pop_24_w_607080.groupby(['base_date','region','emd_nm']).sum()

In [ ]:
g_607080

n_e_resd_24  n_e_work_24  n_e_visit_24
base_date  region emd_nm                                        
2018-01-01 서귀포시_동 남원읍             0.0       0.0000     3513.4114
                  성산읍             0.0     165.8707     4256.0307
                  표선면             0.0     228.0861     3276.4695
           서귀포시_서 대정읍             0.0     166.1916     1810.7644
                  안덕면             0.0     171.0644     3075.9533
...                               ...          ...           ...
2021-06-30 제주시_중  이도2동            0.0     279.2269     3611.7409
                  이호동             0.0      39.8421      746.5294
                  일도1동            0.0      39.8421      308.9147
                  일도2동            0.0     175.1833     1639.5574
                  화북동             0.0     131.7186     1011.9248

[52357 rows x 3 columns]

In [ ]:
g_01020= g_01020.reset_index()
g_304050 = g_304050.reset_index()
g_607080 = g_607080.reset_index()

In [ ]:
g_01020

,base_date,region,emd_nm,n_e_resd_24,n_e_work_24,n_e_visit_24
0,2018-01-01,서귀포시_동,남원읍,0.0,168.7808,8314.9211
1,2018-01-01,서귀포시_동,성산읍,0.0,391.9059,20583.6217
2,2018-01-01,서귀포시_동,표선면,0.0,156.8768,9721.6850
3,2018-01-01,서귀포시_서,대정읍,0.0,342.1410,5068.1368
4,2018-01-01,서귀포시_서,안덕면,0.0,119.0318,9049.8287
...,...,...,...,...,...,...
52352,2021-06-30,제주시_중,이도2동,0.0,655.9708,16139.3463
52353,2021-06-30,제주시_중,이호동,0.0,16.6593,5520.2685
52354,2021-06-30,제주시_중,일도1동,0.0,11.1115,2535.8294
52355,2021-06-30,제주시_중,일도2동,0.0,149.1942,5272.6929


In [ ]:
f_24_w_01020 = g_01020.rename(columns={'n_e_resd_24':'n_e_resd_24_w_01020','n_e_work_24':'n_e_work_24_w_01020','n_e_visit_24':'n_e_visit_24_w_01020'})
f_24_w_01020

,base_date,region,emd_nm,n_e_resd_24_w_01020,n_e_work_24_w_01020,n_e_visit_24_w_01020
0,2018-01-01,서귀포시_동,남원읍,0.0,168.7808,8314.9211
1,2018-01-01,서귀포시_동,성산읍,0.0,391.9059,20583.6217
2,2018-01-01,서귀포시_동,표선면,0.0,156.8768,9721.6850
3,2018-01-01,서귀포시_서,대정읍,0.0,342.1410,5068.1368
4,2018-01-01,서귀포시_서,안덕면,0.0,119.0318,9049.8287
...,...,...,...,...,...,...
52352,2021-06-30,제주시_중,이도2동,0.0,655.9708,16139.3463
52353,2021-06-30,제주시_중,이호동,0.0,16.6593,5520.2685
52354,2021-06-30,제주시_중,일도1동,0.0,11.1115,2535.8294
52355,2021-06-30,제주시_중,일도2동,0.0,149.1942,5272.6929


In [ ]:
f_24_w_304050 = g_304050.rename(columns={'n_e_resd_24':'n_e_resd_24_w_304050','n_e_work_24':'n_e_work_24_w_304050','n_e_visit_24':'n_e_visit_24_w_304050'})
f_24_w_607080 = g_607080.rename(columns={'n_e_resd_24':'n_e_resd_24_w_607080','n_e_work_24':'n_e_work_24_w_607080','n_e_visit_24':'n_e_visit_24_w_607080'})

In [ ]:
f_24_w_607080

,base_date,region,emd_nm,n_e_resd_24_w_607080,n_e_work_24_w_607080,n_e_visit_24_w_607080
0,2018-01-01,서귀포시_동,남원읍,0.0,0.0000,3513.4114
1,2018-01-01,서귀포시_동,성산읍,0.0,165.8707,4256.0307
2,2018-01-01,서귀포시_동,표선면,0.0,228.0861,3276.4695
3,2018-01-01,서귀포시_서,대정읍,0.0,166.1916,1810.7644
4,2018-01-01,서귀포시_서,안덕면,0.0,171.0644,3075.9533
...,...,...,...,...,...,...
52352,2021-06-30,제주시_중,이도2동,0.0,279.2269,3611.7409
52353,2021-06-30,제주시_중,이호동,0.0,39.8421,746.5294
52354,2021-06-30,제주시_중,일도1동,0.0,39.8421,308.9147
52355,2021-06-30,제주시_중,일도2동,0.0,175.1833,1639.5574


#### 3-2 남성 나이대별 컬럼생성
- 0-10대 , 10-20대,  20-30대 : 청년층 ->01020
-  30-40대 , 40-50대,  50-60대 : 중년층 ->304050
- 60-70대 , 70-80대,  80대 이상 : 노년층 ->607080

In [ ]:
pop_24_m_01020= pop_24_m[(pop_24_m['age']==0)|(pop_24_m['age']==10)|(pop_24_m['age']==20)]
pop_24_m_304050= pop_24_m[(pop_24_m['age']==30)|(pop_24_m['age']==40)|(pop_24_m['age']==50)]
pop_24_m_607080= pop_24_m[(pop_24_m['age']==60)|(pop_24_m['age']==70)|(pop_24_m['age']==80)]

In [ ]:
pop_24_m_01020

,base_date,emd_nm,age,n_e_resd_24,n_e_work_24,n_e_visit_24,region
0,2018-01-01,건입동,0,0.0,3.7485,112.4523,제주시_중
1,2018-01-01,건입동,10,0.0,4.5077,135.2296,제주시_중
2,2018-01-01,건입동,20,0.0,13.2166,287.4608,제주시_중
18,2018-01-01,구좌읍,0,0.0,3.7485,164.9301,제주시_동
19,2018-01-01,구좌읍,10,0.0,4.5077,198.3368,제주시_동
...,...,...,...,...,...,...,...
10774832,2021-06-30,화북동,10,0.0,4.5128,139.8981,제주시_중
10774833,2021-06-30,화북동,20,0.0,11.1999,187.5990,제주시_중
10774849,2021-06-30,효돈동,0,0.0,0.0000,34.0820,서귀포시_중
10774850,2021-06-30,효돈동,10,0.0,0.0000,40.6156,서귀포시_중


In [ ]:
pop_24_m_01020= pop_24_m_01020.drop(['age'],axis=1)
pop_24_m_304050= pop_24_m_304050.drop(['age'],axis=1)
pop_24_m_607080= pop_24_m_607080.drop(['age'],axis=1)

In [ ]:
g_01020 = pop_24_m_01020.groupby(['base_date','region','emd_nm']).sum()
g_304050 = pop_24_m_304050.groupby(['base_date','region','emd_nm']).sum()
g_607080 = pop_24_m_607080.groupby(['base_date','region','emd_nm']).sum()

In [ ]:
g_01020 = g_01020.reset_index()
g_304050 = g_304050.reset_index()
g_607080 = g_607080.reset_index()

In [ ]:
g_01020

,base_date,region,emd_nm,n_e_resd_24,n_e_work_24,n_e_visit_24
0,2018-01-01,서귀포시_동,남원읍,0.0,339.2087,8304.6658
1,2018-01-01,서귀포시_동,성산읍,0.0,367.3045,17202.1982
2,2018-01-01,서귀포시_동,표선면,0.0,344.2965,8101.8225
3,2018-01-01,서귀포시_서,대정읍,0.0,378.9049,3982.0264
4,2018-01-01,서귀포시_서,안덕면,0.0,378.8563,7988.2868
...,...,...,...,...,...,...
52352,2021-06-30,제주시_중,이도2동,0.0,553.6133,15321.2889
52353,2021-06-30,제주시_중,이호동,0.0,14.0199,4475.5807
52354,2021-06-30,제주시_중,일도1동,0.0,8.4171,1622.2702
52355,2021-06-30,제주시_중,일도2동,0.0,297.8015,5066.2934


In [ ]:
f_24_m_01020 = g_01020.rename(columns={'n_e_resd_24':'n_e_resd_24_m_01020','n_e_work_24':'n_e_work_24_m_01020','n_e_visit_24':'n_e_visit_24_m_01020'})
f_24_m_01020

,base_date,region,emd_nm,n_e_resd_24_m_01020,n_e_work_24_m_01020,n_e_visit_24_m_01020
0,2018-01-01,서귀포시_동,남원읍,0.0,339.2087,8304.6658
1,2018-01-01,서귀포시_동,성산읍,0.0,367.3045,17202.1982
2,2018-01-01,서귀포시_동,표선면,0.0,344.2965,8101.8225
3,2018-01-01,서귀포시_서,대정읍,0.0,378.9049,3982.0264
4,2018-01-01,서귀포시_서,안덕면,0.0,378.8563,7988.2868
...,...,...,...,...,...,...
52352,2021-06-30,제주시_중,이도2동,0.0,553.6133,15321.2889
52353,2021-06-30,제주시_중,이호동,0.0,14.0199,4475.5807
52354,2021-06-30,제주시_중,일도1동,0.0,8.4171,1622.2702
52355,2021-06-30,제주시_중,일도2동,0.0,297.8015,5066.2934


In [ ]:
f_24_m_304050 = g_304050.rename(columns={'n_e_resd_24':'n_e_resd_24_m_304050','n_e_work_24':'n_e_work_24_m_304050','n_e_visit_24':'n_e_visit_24_m_304050'})
f_24_m_607080 = g_607080.rename(columns={'n_e_resd_24':'n_e_resd_24_m_607080','n_e_work_24':'n_e_work_24_m_607080','n_e_visit_24':'n_e_visit_24_m_607080'})

In [ ]:
f_24_m_607080

,base_date,region,emd_nm,n_e_resd_24_m_607080,n_e_work_24_m_607080,n_e_visit_24_m_607080
0,2018-01-01,서귀포시_동,남원읍,0.0,193.0342,2585.7948
1,2018-01-01,서귀포시_동,성산읍,0.0,276.3276,3915.3667
2,2018-01-01,서귀포시_동,표선면,0.0,421.7818,3173.9354
3,2018-01-01,서귀포시_서,대정읍,0.0,289.8330,1625.0470
4,2018-01-01,서귀포시_서,안덕면,0.0,403.8552,2717.3990
...,...,...,...,...,...,...
52352,2021-06-30,제주시_중,이도2동,0.0,892.3953,4979.8951
52353,2021-06-30,제주시_중,이호동,0.0,118.9187,1311.5173
52354,2021-06-30,제주시_중,일도1동,0.0,69.9850,551.7238
52355,2021-06-30,제주시_중,일도2동,0.0,666.1039,3256.2192


### 4) 낮 시간대 데이터 성별 및 나이 전처리

In [ ]:
pop_12 =  pd.read_csv('./datas/내국인유동인구_그외_낮시간대_데이터이동용.csv', encoding='CP949')
pop_12.head()

,base_date,emd_nm,sex,age,n_e_resd_12,n_e_work_12,n_e_visit_12,region
0,2018-01-01,건입동,남성,0,0.0,3.7493,108.7295,제주시_중
1,2018-01-01,건입동,남성,10,0.0,4.5087,130.7527,제주시_중
2,2018-01-01,건입동,남성,20,0.0,9.9194,267.8237,제주시_중
3,2018-01-01,건입동,남성,30,0.0,3.2137,241.0283,제주시_중
4,2018-01-01,건입동,남성,40,0.0,10.4405,236.6507,제주시_중


In [ ]:
pop_12['emd_nm'].unique()

array(['건입동', '구좌읍', '남원읍', '노형동', '대륜동', '대정읍', '대천동', '도두동', '동홍동',
       '봉개동', '삼도1동', '삼도2동', '삼양동', '서홍동', '성산읍', '송산동', '아라동', '안덕면',
       '애월읍', '연동', '영천동', '예래동', '오라동', '외도동', '용담1동', '용담2동', '우도면',
       '이도1동', '이도2동', '이호동', '일도1동', '일도2동', '정방동', '조천읍', '중문동', '중앙동',
       '천지동', '추자면', '표선면', '한경면', '한림읍', '화북동', '효돈동'], dtype=object)

#### 4-0 성별 나누기

In [ ]:
pop_12_w = pop_12[pop_12['sex']=='여성']
pop_12_m = pop_12[pop_12['sex']=='남성']

In [ ]:
pop_12_w= pop_12_w.drop(['sex'],axis=1)
pop_12_m=pop_12_m.drop(['sex'],axis=1)

#### 4-1 여성 나이대별 컬럼생성
- 0-10대 , 10-20대,  20-30대 : 청년층 ->01020
-  30-40대 , 40-50대,  50-60대 : 중년층 ->304050
- 60-70대 , 70-80대,  80대 이상 : 노년층 ->607080

In [ ]:
pop_12_w_01020= pop_12_w[(pop_12_w['age']==0)|(pop_12_w['age']==10)|(pop_12_w['age']==20)]
pop_12_w_304050= pop_12_w[(pop_12_w['age']==30)|(pop_12_w['age']==40)|(pop_12_w['age']==50)]
pop_12_w_607080= pop_12_w[(pop_12_w['age']==60)|(pop_12_w['age']==70)|(pop_12_w['age']==80)]

In [ ]:
pop_12_w_304050

,base_date,emd_nm,age,n_e_resd_12,n_e_work_12,n_e_visit_12,region
10,2018-01-01,건입동,30,0.0,0.0000,217.5086,제주시_중
11,2018-01-01,건입동,40,0.0,3.4802,247.0962,제주시_중
12,2018-01-01,건입동,50,0.0,10.8299,194.9385,제주시_중
28,2018-01-01,구좌읍,30,0.0,0.0000,621.9387,제주시_동
29,2018-01-01,구좌읍,40,0.0,10.4407,431.5484,제주시_동
...,...,...,...,...,...,...,...
12754147,2021-06-30,화북동,40,0.0,31.5564,133.2382,제주시_중
12754148,2021-06-30,화북동,50,0.0,15.3481,135.0632,제주시_중
12754164,2021-06-30,효돈동,30,0.0,0.0000,72.2002,서귀포시_중
12754165,2021-06-30,효돈동,40,0.0,0.0000,59.6066,서귀포시_중


In [ ]:
pop_12_w_01020= pop_12_w_01020.drop(['age'],axis=1)
pop_12_w_304050= pop_12_w_304050.drop(['age'],axis=1)
pop_12_w_607080= pop_12_w_607080.drop(['age'],axis=1)

In [ ]:
g_01020 = pop_12_w_01020.groupby(['base_date','region','emd_nm']).sum()
g_304050 = pop_12_w_304050.groupby(['base_date','region','emd_nm']).sum()
g_607080 = pop_12_w_607080.groupby(['base_date','region','emd_nm']).sum()

In [ ]:
g_01020 = g_01020.reset_index()
g_304050 = g_304050.reset_index()
g_607080 = g_607080.reset_index()

In [ ]:
g_01020

,base_date,region,emd_nm,n_e_resd_12,n_e_work_12,n_e_visit_12
0,2018-01-01,서귀포시_동,남원읍,0.0,238.4790,11539.0598
1,2018-01-01,서귀포시_동,성산읍,0.0,419.6024,30581.4812
2,2018-01-01,서귀포시_동,표선면,0.0,180.5146,11066.0683
3,2018-01-01,서귀포시_서,대정읍,0.0,463.6985,7274.5049
4,2018-01-01,서귀포시_서,안덕면,0.0,133.9160,15648.2139
...,...,...,...,...,...,...
52352,2021-06-30,제주시_중,이도2동,0.0,822.6223,18501.9023
52353,2021-06-30,제주시_중,이호동,0.0,63.8746,6272.5477
52354,2021-06-30,제주시_중,일도1동,0.0,47.2122,2478.7846
52355,2021-06-30,제주시_중,일도2동,0.0,192.6015,6334.9247


In [ ]:
f_12_w_01020= g_01020.rename(columns={'n_e_resd_12':'n_e_resd_12_w_01020','n_e_work_12':'n_e_work_12_w_01020','n_e_visit_12':'n_e_visit_12_w_01020'})
f_12_w_01020

,base_date,region,emd_nm,n_e_resd_12_w_01020,n_e_work_12_w_01020,n_e_visit_12_w_01020
0,2018-01-01,서귀포시_동,남원읍,0.0,238.4790,11539.0598
1,2018-01-01,서귀포시_동,성산읍,0.0,419.6024,30581.4812
2,2018-01-01,서귀포시_동,표선면,0.0,180.5146,11066.0683
3,2018-01-01,서귀포시_서,대정읍,0.0,463.6985,7274.5049
4,2018-01-01,서귀포시_서,안덕면,0.0,133.9160,15648.2139
...,...,...,...,...,...,...
52352,2021-06-30,제주시_중,이도2동,0.0,822.6223,18501.9023
52353,2021-06-30,제주시_중,이호동,0.0,63.8746,6272.5477
52354,2021-06-30,제주시_중,일도1동,0.0,47.2122,2478.7846
52355,2021-06-30,제주시_중,일도2동,0.0,192.6015,6334.9247


In [ ]:
f_12_w_304050 = g_304050.rename(columns={'n_e_resd_12':'n_e_resd_12_w_304050','n_e_work_12':'n_e_work_12_w_304050','n_e_visit_12':'n_e_visit_12_w_304050'})
f_12_w_607080 = g_607080.rename(columns={'n_e_resd_12':'n_e_resd_12_w_607080','n_e_work_12':'n_e_work_12_w_607080','n_e_visit_12':'n_e_visit_12_w_607080'})

In [ ]:
f_24_w_304050

,base_date,region,emd_nm,n_e_resd_24_w_304050,n_e_work_24_w_304050,n_e_visit_24_w_304050
0,2018-01-01,서귀포시_동,남원읍,0.0,428.2292,11621.5075
1,2018-01-01,서귀포시_동,성산읍,0.0,491.1014,23389.4367
2,2018-01-01,서귀포시_동,표선면,0.0,407.7292,14276.8166
3,2018-01-01,서귀포시_서,대정읍,0.0,184.5390,5937.4816
4,2018-01-01,서귀포시_서,안덕면,0.0,181.6826,12432.1039
...,...,...,...,...,...,...
52352,2021-06-30,제주시_중,이도2동,0.0,537.7482,7275.2894
52353,2021-06-30,제주시_중,이호동,0.0,67.4080,2110.0370
52354,2021-06-30,제주시_중,일도1동,0.0,47.7891,1043.5580
52355,2021-06-30,제주시_중,일도2동,0.0,237.2438,2735.2854


#### 4-2 남성 나이대별 컬럼생성
- 0-10대 , 10-20대,  20-30대 : 청년층 ->01020
-  30-40대 , 40-50대,  50-60대 : 중년층 ->304050
- 60-70대 , 70-80대,  80대 이상 : 노년층 ->607080

In [ ]:
pop_12_m_01020= pop_12_m[(pop_12_m['age']==0)|(pop_12_m['age']==10)|(pop_12_m['age']==20)]
pop_12_m_304050= pop_12_m[(pop_12_m['age']==30)|(pop_12_m['age']==40)|(pop_12_m['age']==50)]
pop_12_m_607080= pop_12_m[(pop_12_m['age']==60)|(pop_12_m['age']==70)|(pop_12_m['age']==80)]

In [ ]:
pop_12_m_304050

,base_date,emd_nm,age,n_e_resd_12,n_e_work_12,n_e_visit_12,region
3,2018-01-01,건입동,30,0.0,3.2137,241.0283,제주시_중
4,2018-01-01,건입동,40,0.0,10.4405,236.6507,제주시_중
5,2018-01-01,건입동,50,0.0,3.7111,218.9537,제주시_중
19,2018-01-01,구좌읍,30,0.0,12.8548,414.5688,제주시_동
20,2018-01-01,구좌읍,40,0.0,6.9603,389.7777,제주시_동
...,...,...,...,...,...,...,...
12754138,2021-06-30,화북동,40,0.0,19.4684,155.7472,제주시_중
12754139,2021-06-30,화북동,50,0.0,38.2011,187.8218,제주시_중
12754155,2021-06-30,효돈동,30,0.0,0.0000,62.5348,서귀포시_중
12754156,2021-06-30,효돈동,40,0.0,6.4895,48.6710,서귀포시_중


In [ ]:
pop_12_m_01020= pop_12_m_01020.drop(['age'],axis=1)
pop_12_m_304050= pop_12_m_304050.drop(['age'],axis=1)
pop_12_m_607080= pop_12_m_607080.drop(['age'],axis=1)

In [ ]:
g_01020 = pop_12_m_01020.groupby(['base_date','region','emd_nm']).sum()
g_304050 = pop_12_m_304050.groupby(['base_date','region','emd_nm']).sum()
g_607080 = pop_12_m_607080.groupby(['base_date','region','emd_nm']).sum()

In [ ]:
g_01020 = g_01020.reset_index()
g_304050 = g_304050.reset_index()
g_607080 = g_607080.reset_index()

In [ ]:
f_12_m_01020 = g_01020.rename(columns={'n_e_resd_12':'n_e_resd_12_m_01020','n_e_work_12':'n_e_work_12_m_01020','n_e_visit_12':'n_e_visit_12_m_01020'})
f_12_m_01020

,base_date,region,emd_nm,n_e_resd_12_m_01020,n_e_work_12_m_01020,n_e_visit_12_m_01020
0,2018-01-01,서귀포시_동,남원읍,0.0,389.8104,10443.0953
1,2018-01-01,서귀포시_동,성산읍,0.0,267.4227,26124.6164
2,2018-01-01,서귀포시_동,표선면,0.0,364.8953,9423.1256
3,2018-01-01,서귀포시_서,대정읍,0.0,427.4759,5936.6970
4,2018-01-01,서귀포시_서,안덕면,0.0,381.5949,13360.8733
...,...,...,...,...,...,...
52352,2021-06-30,제주시_중,이도2동,0.0,629.6329,16316.2000
52353,2021-06-30,제주시_중,이호동,0.0,36.4622,4077.4549
52354,2021-06-30,제주시_중,일도1동,0.0,2.8047,1719.7892
52355,2021-06-30,제주시_중,일도2동,0.0,283.6146,5608.5362


In [ ]:
f_12_m_304050 = g_304050.rename(columns={'n_e_resd_12':'n_e_resd_12_m_304050','n_e_work_12':'n_e_work_12_m_304050','n_e_visit_12':'n_e_visit_12_m_304050'})
f_12_m_607080 = g_607080.rename(columns={'n_e_resd_12':'n_e_resd_12_m_607080','n_e_work_12':'n_e_work_12_m_607080','n_e_visit_12':'n_e_visit_12_m_607080'})

In [ ]:
f_12_m_304050

,base_date,region,emd_nm,n_e_resd_12_m_304050,n_e_work_12_m_304050,n_e_visit_12_m_304050
0,2018-01-01,서귀포시_동,남원읍,0.0,965.1529,15785.0081
1,2018-01-01,서귀포시_동,성산읍,0.0,1541.1003,34305.7064
2,2018-01-01,서귀포시_동,표선면,0.0,995.8757,15170.4908
3,2018-01-01,서귀포시_서,대정읍,0.0,796.3328,10307.6216
4,2018-01-01,서귀포시_서,안덕면,0.0,1028.2140,21441.7286
...,...,...,...,...,...,...
52352,2021-06-30,제주시_중,이도2동,0.0,1456.9243,12545.3250
52353,2021-06-30,제주시_중,이호동,0.0,119.2511,4386.4481
52354,2021-06-30,제주시_중,일도1동,0.0,84.0050,1807.1456
52355,2021-06-30,제주시_중,일도2동,0.0,270.8036,5110.6937


### 5) 각 시간대별 성별 나이별 dataframe merge


In [ ]:
merge_data = pd.merge(f_12_w_01020,f_12_w_304050, how='outer',on=['base_date','region','emd_nm'])

In [ ]:
merge_data = pd.merge(merge_data,f_12_w_607080, how='outer',on=['base_date','region','emd_nm'])

In [ ]:
merge_data = pd.merge(merge_data,f_12_m_01020, how='outer',on=['base_date','region','emd_nm'])


In [ ]:
merge_data = pd.merge(merge_data,f_12_m_304050, how='outer',on=['base_date','region','emd_nm'])

In [ ]:
merge_data = pd.merge(merge_data,f_12_m_607080, how='outer',on=['base_date','region','emd_nm'])

In [ ]:
merge_data = pd.merge(merge_data,f_24_w_01020, how='outer',on=['base_date','region','emd_nm'])

In [ ]:
merge_data = pd.merge(merge_data,f_24_w_304050, how='outer',on=['base_date','region','emd_nm'])

In [ ]:
merge_data = pd.merge(merge_data,f_24_w_607080, how='outer',on=['base_date','region','emd_nm'])

In [ ]:
merge_data = pd.merge(merge_data,f_24_m_01020, how='outer',on=['base_date','region','emd_nm'])

In [ ]:
merge_data = pd.merge(merge_data,f_24_m_304050, how='outer',on=['base_date','region','emd_nm'])

In [ ]:
merge_data = pd.merge(merge_data,f_24_m_607080, how='outer',on=['base_date','region','emd_nm'])

### 6) 데이터 저장하기

In [ ]:
merge_data.to_csv("./datas/newdata_내국인유동인구_그외지역.csv", index=False, encoding='CP949')